In [93]:
%pip install BeautifulSoup4 
%pip install gensim nltk pandas seaborn keras
%pip install -U scikit-learn
%pip install tensorflow
%pip install emoji
%pip install demoji


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.

Note: you may need to restart the kernel to use updated packages.


In [95]:
import re 
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('floresta')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk import pos_tag, word_tokenize
import joblib
import sys  
import demoji

demoji.download_codes()

def remove_emojis(text):
    demoji.download_codes()  # Download the emoji codes (run this once)

    text_without_emojis = demoji.replace(text, '')
    return text_without_emojis


def clean_text(text):
  text = ' '.join([ word for word in text.split(' ') if not word.startswith('@') ])
  text = re.sub(r"[^A-Za-z ]+", '', text) # keep only letters and spaces
  text = text.strip()
  return text

def remove_stop_words(text):
  stopwords_pt = stopwords.words('spanish')
  text_without_sw = [word for word in text.split(' ') if not word in stopwords_pt]
  return (" ").join(text_without_sw)

def lemmatization_nltk(text):
  lemmatizer = WordNetLemmatizer()
  words = nltk.word_tokenize(text, language='spanish')
  lemmas = [lemmatizer.lemmatize(p).lower() for p in words]
  return (" ").join(lemmas)


def remove_proper_nouns(text):
    words = []
    for word, tag in pos_tag(word_tokenize(text)):
        if tag != 'NNP' and tag != 'NNPS':  # Check for proper nouns tags
            words.append(word)
    return ' '.join(words)


def remove_urls(text):
    # Regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'

    # Remove URLs from the text using the pattern
    text_without_urls = re.sub(url_pattern, '', text)

    return text_without_urls

def normalize_text(text):
  text = remove_urls(text)
  text = remove_emojis(text)
  text = clean_text(text)
  text = remove_proper_nouns(text)
  text = text.lower() # outside clean_text because capitalization influences remove_proper_nouns function
  text = remove_stop_words(text)
  text = lemmatization_nltk(text)

  return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package floresta to
[nltk_data]     C:\Users\Maria\AppData\Roaming\nltk_data...
[nltk_data]   Package floresta is already up-to-date!
C:\Users\Maria\AppData\Local\Temp\ipykernel_27764\3588849776.py:15: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


# Scarpe tweets from user

In [132]:
import requests
from bs4 import BeautifulSoup
import time

def scrape_tweets(url, num_tweets):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Referer': 'https://nitter.net/',
    'Accept-Language': 'en-US,en;q=0.9',
    }
    all_tweets = []

    while url and len(all_tweets) < num_tweets:
        response = requests.get(url, headers=headers)
        print(response)

        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            tweet_elements = soup.find_all("div", class_="tweet-content media-body")

            for tweet_element in tweet_elements:
                tweet_text = tweet_element.get_text(strip=True)
                
                # Skip empty tweets
                if tweet_text:
                    all_tweets.append(tweet_text)
                    print(len(all_tweets), tweet_text)
                    
                    if len(all_tweets) >= num_tweets:
                        break

            # Find the URL for the next page based on the "Load more" button
            load_more_link = soup.select_one("div.show-more a[href^='?cursor=']")
            print(load_more_link)
            if load_more_link:
                next_page_url = load_more_link["href"]
                url = f"https://nitter.net/{username}{next_page_url}"
                print(url)
            else:
                url = None  # No more pages

            # Add a delay to avoid rate limiting
            time.sleep(1)

    return all_tweets

# URL of the Nitter profile page
#username = "IbaiLlanos"
username = "eldiariodedross"
url = f"https://nitter.net/{username}"
print(url)

num_tweets_to_scrape = 100
tweets = scrape_tweets(url, num_tweets_to_scrape)

print("Extracted Tweets:")
for tweet in tweets:
    print(tweet)

https://nitter.net/eldiariodedross
<Response [200]>
1 A ciertos influencers del 1er mundo les da ansiedad venir a Latinoamérica porque creen que los van a apuñalar. Pero tienen cara de que no les molesta que un tercermundista le meta una puñalada de carne a la novia.
2 Sé que hay personas que se sienten ofendidas por mi postura anti-comunista. Sepan que se me hacen igual de asquerosos e idiotas que un nazi.
3 Feliz domingo ☕️
4 JAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAAJAJAJAJAJAJA
5 Dylan Mulvaney: primero fue gay, luego dijo que se identificaba como una niña. Después se identificó
como mujer. Más tarde dijo ser lesbiana. Y ahora quiere "embarazarse". No necesita que se le celebre... necesita que se le ayude.
6 Veo comunistas rebuznando que no midamos a sus próceres de mierda con la misma vara que ellos miden a Bolívar, Roca, Franco, Pinochet, Reagan, etc.
7 ¿Alguien sabe dónde queda? Es para un video………
8 No necesitas la aprobación ni la admiración de nadie. Y 

In [133]:
# Specify the file path where you want to save the tweets
output_file_path = "tweets.txt"

# Save the extracted tweets to a text file
with open(output_file_path, "w", encoding="utf-8") as file:
    for tweet in tweets:
        file.write(tweet + "\n")

print(f"Extracted {len(tweets)} tweets and saved them to '{output_file_path}'.")

Extracted 100 tweets and saved them to 'tweets.txt'.


# Normalize the data

In [134]:
import numpy as np
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
# cálculo de toxicidade 
tweets_normalized = [normalize_text(tweet) for tweet in tweets]
tweets_tokens = pad_sequences(tokenizer.texts_to_sequences(tweets_normalized), maxlen = 30)  
print(tweets_normalized)

['A ciertos influencers del 1er mundo les da ansiedad venir a Latinoamérica porque creen que los van a apuñalar. Pero tienen cara de que no les molesta que un tercermundista le meta una puñalada de carne a la novia.', 'Sé que hay personas que se sienten ofendidas por mi postura anti-comunista. Sepan que se me hacen igual de asquerosos e idiotas que un nazi.', 'Feliz domingo ☕️', 'JAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAAJAJAJAJAJAJA', 'Dylan Mulvaney: primero fue gay, luego dijo que se identificaba como una niña. Después se identificó\ncomo mujer. Más tarde dijo ser lesbiana. Y ahora quiere "embarazarse". No necesita que se le celebre... necesita que se le ayude.', 'Veo comunistas rebuznando que no midamos a sus próceres de mierda con la misma vara que ellos miden a Bolívar, Roca, Franco, Pinochet, Reagan, etc.', '¿Alguien sabe dónde queda? Es para un video………', 'No necesitas la aprobación ni la admiración de nadie. Y mucho menos de gente que está por debajo de 

# User BERT to predict Toxcicity

bert-base-spanish

In [141]:
import os
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
import csv


# Load the saved model
model_path = "bert_spanish_28A_4/model.pth"
model_name = "dccuchile/bert-base-spanish-wwm-uncased"

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
model.load_state_dict(torch.load(model_path))
model.eval()

# Prepare the tokenized input for BERT
tweets_normalized = [normalize_text(tweet) for tweet in tweets]
encoded_inputs = tokenizer(tweets_normalized, padding='longest', truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Use the BERT model to make predictions
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask).logits

# Convert logits to probabilities using sigmoid
probs = torch.sigmoid(logits).cpu().numpy()

# Define the toxicity ranges
toxicity_ranges = {
    0: (0, 25),    # label 0: Non-Offensive
    1: (25, 50),   # label 1: Non-Offensive but with Expletive Language
    2: (50, 100),  # label 2: Offensive Targeting a Group
    3: (50, 100)   # label 3: Offensive Targeting an Individual
}

toxicity_ranges_next = {
(0, 10)   # label 3: Offensive Targeting an Individual
}


# Initialize a list to store toxicity percentages
toxicity_percentages = []

# Process each tweet's probabilities
for prob in probs:
    top_indices = np.argsort(prob)[-2:]
    label = np.argmax(prob)  # Get the predicted label
    #print("Predicted Probabilities for Each Label:", prob)
    #print("Predicted label for Each Label:", top_indices)
    #print("Predicted label for Each Label:", label)
    toxicity_range = toxicity_ranges[label]  # Get the toxicity range for the predicted label
    #print(toxicity_range) #
    # Map the predicted probability to the corresponding toxicity range
    mapped_probability = np.interp(prob[label], [0, 1], toxicity_range)
    mapped_probability_next = np.interp(prob[top_indices[0]], [0, 1], (0,10))
    #print(mapped_probability, top_indices[0], mapped_probability_next) # Print
    # Calculate the toxicity percentage within the mapped range
    toxicity_percentage = (mapped_probability - mapped_probability_next)
    #print("Pepe",toxicity_percentage)
    toxicity_percentages.append(toxicity_percentage)
total_toxicity_score= np.mean(toxicity_percentages)
print("Total Toxicity Percentage BERT Spanish: ",total_toxicity_score)
print("----------------------------------------------------\n")
for i, tweet in enumerate(tweets):
    label = np.argmax(probs[i])  # Get the predicted label for this tweet
    print(f"Tweet: {tweet}")
    print(f"Predicted Label: {label}")
    print(f"Toxicity Percentages: {toxicity_percentages[i]}%\n")
    print("----------------------------------------------------")

# Define the output file name
output_file = "toxicity_results_bert_spanish_28A_eldiariodedross.txt"

# Open the file for writing
# Open the file for writing with 'utf-8' encoding
with open(output_file, "w", encoding="utf-8") as f:
    f.write(f"Total Toxicity Percentage BERT Spanish: {total_toxicity_score}\n")
    f.write("----------------------------------------------------\n")
    for i, tweet in enumerate(tweets):
        label = np.argmax(probs[i])  # Get the predicted label for this tweet
        f.write(f"Tweet: {tweet}\n")
        f.write(f"Predicted Label: {label}\n")
        f.write(f"Toxicity Percentages: {toxicity_percentages[i]}%\n")
        f.write("----------------------------------------------------\n")

print("Results saved to", output_file)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Toxicity Percentage BERT Spanish:  45.70362723469734
----------------------------------------------------

Tweet: A ciertos influencers del 1er mundo les da ansiedad venir a Latinoamérica porque creen que los van a apuñalar. Pero tienen cara de que no les molesta que un tercermundista le meta una puñalada de carne a la novia.
Predicted Label: 2
Toxicity Percentages: 93.33722203969955%

----------------------------------------------------
Tweet: Sé que hay personas que se sienten ofendidas por mi postura anti-comunista. Sepan que se me hacen igual de asquerosos e idiotas que un nazi.
Predicted Label: 2
Toxicity Percentages: 93.51439118385315%

----------------------------------------------------
Tweet: Feliz domingo ☕️
Predicted Label: 0
Toxicity Percentages: 17.139700055122375%

----------------------------------------------------
Tweet: JAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAAJAJAJAJAJAJA
Predicted Label: 1
Toxicity Percentages: 45.67947208881378%

-----

In [ ]:
# Create a list to store the data for the CSV
csv_data = [("tweet", "predicted Label", "toxicity percentage")]

# Process each tweet's probabilities
for i, prob in enumerate(probs):
    top_indices = np.argsort(prob)[-2:]
    label = np.argmax(prob)  # Get the predicted label
    toxicity_range = toxicity_ranges[label]  # Get the toxicity range for the predicted label
    mapped_probability = np.interp(prob[label], [0, 1], toxicity_range)
    mapped_probability_next = np.interp(prob[top_indices[0]], [0, 1], (0, 10))
    toxicity_percentage = (mapped_probability - mapped_probability_next)
    toxicity_percentages.append(toxicity_percentage)

    # Append the data to the CSV list
    csv_data.append((tweets[i], label, toxicity_percentage))

total_toxicity_score = np.mean(toxicity_percentages)

# Save the data to a CSV file with 'utf-8' encoding
csv_file = "toxicity_resultsbert_spanish_28A_eldiariodedross.csv"

with open(csv_file, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print("Results saved to", csv_file)

bert-base-multilingual

In [142]:
import os
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification


# Load the saved model
model_path = "bert_multi_30A_10/model.pth"
model_name = "bert-base-multilingual-uncased"

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
model.load_state_dict(torch.load(model_path))
model.eval()

# Prepare the tokenized input for BERT
tweets_normalized = [normalize_text(tweet) for tweet in tweets]
encoded_inputs = tokenizer(tweets_normalized, padding='longest', truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Use the BERT model to make predictions
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask).logits

# Convert logits to probabilities using sigmoid
probs = torch.sigmoid(logits).cpu().numpy()

# Define the toxicity ranges
toxicity_ranges = {
    0: (0, 25),    # label 0: Non-Offensive
    1: (25, 50),   # label 1: Non-Offensive but with Expletive Language
    2: (50, 100),  # label 2: Offensive Targeting a Group
    3: (50, 100)   # label 3: Offensive Targeting an Individual
}

toxicity_ranges_next = {
(0, 10)   # label 3: Offensive Targeting an Individual
}


# Initialize a list to store toxicity percentages
toxicity_percentages = []

# Process each tweet's probabilities
for prob in probs:
    top_indices = np.argsort(prob)[-2:]
    label = np.argmax(prob)  # Get the predicted label
    #print("Predicted Probabilities for Each Label:", prob)
    #print("Predicted label for Each Label:", top_indices)
    #print("Predicted label for Each Label:", label)
    toxicity_range = toxicity_ranges[label]  # Get the toxicity range for the predicted label
    #print(toxicity_range) #
    # Map the predicted probability to the corresponding toxicity range
    mapped_probability = np.interp(prob[label], [0, 1], toxicity_range)
    mapped_probability_next = np.interp(prob[top_indices[0]], [0, 1], (0,10))
    #print(mapped_probability, top_indices[0], mapped_probability_next) # Print
    # Calculate the toxicity percentage within the mapped range
    toxicity_percentage = (mapped_probability - mapped_probability_next)
    #print("Pepe",toxicity_percentage)
    toxicity_percentages.append(toxicity_percentage)
total_toxicity_score= np.mean(toxicity_percentages)
print("Total Toxicity Percentage BERT multi: ",total_toxicity_score)
print("----------------------------------------------------\n")
for i, tweet in enumerate(tweets):
    label = np.argmax(probs[i])  # Get the predicted label for this tweet
    print(f"Tweet: {tweet}")
    print(f"Predicted Label: {label}")
    print(f"Toxicity Percentages: {toxicity_percentages[i]}%\n")
    print("----------------------------------------------------")



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Toxicity Percentage BERT multi:  40.107630245387554
----------------------------------------------------

Tweet: A ciertos influencers del 1er mundo les da ansiedad venir a Latinoamérica porque creen que los van a apuñalar. Pero tienen cara de que no les molesta que un tercermundista le meta una puñalada de carne a la novia.
Predicted Label: 2
Toxicity Percentages: 97.11857587099075%

----------------------------------------------------
Tweet: Sé que hay personas que se sienten ofendidas por mi postura anti-comunista. Sepan que se me hacen igual de asquerosos e idiotas que un nazi.
Predicted Label: 2
Toxicity Percentages: 96.3567328453064%

----------------------------------------------------
Tweet: Feliz domingo ☕️
Predicted Label: 0
Toxicity Percentages: 20.663845241069794%

----------------------------------------------------
Tweet: JAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAJAAJAJAJAJAJAJA
Predicted Label: 1
Toxicity Percentages: 43.25358361005783%

-------

In [143]:
# Initialize a list to store toxicity percentages
toxicity_percentages = []
# Create a list to store the data for the CSV
csv_data = [("tweet", "predicted Label", "toxicity percentage")]

# Process each tweet's probabilities
for i, prob in enumerate(probs):
    top_indices = np.argsort(prob)[-2:]
    label = np.argmax(prob)  # Get the predicted label
    toxicity_range = toxicity_ranges[label]  # Get the toxicity range for the predicted label
    mapped_probability = np.interp(prob[label], [0, 1], toxicity_range)
    mapped_probability_next = np.interp(prob[top_indices[0]], [0, 1], (0, 10))
    toxicity_percentage = (mapped_probability - mapped_probability_next)
    toxicity_percentages.append(toxicity_percentage)

    # Append the data to the CSV list
    csv_data.append((tweets[i], label, toxicity_percentage))


total_toxicity_score = np.mean(toxicity_percentages)
print("Total Toxicity Percentage BERT multi: ",total_toxicity_score)
print("----------------------------------------------------\n")
# Save the data to a CSV file
csv_file = "toxicity_results.csv"

with open(csv_file, mode="w", newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print("Results saved to", csv_file)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Toxicity Percentage BERT multi:  40.107630245387554
----------------------------------------------------

Results saved to toxicity_results.csv


In [144]:
import os
import torch
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification

# Define a function to normalize text (you can replace this with your actual normalization function)
def normalize_text(text):
    # Replace this with your text normalization logic
    return text

# Load the saved model
model_path = "bert_multi_30A_10/model.pth"
model_name = "bert-base-multilingual-uncased"

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=4)
model.load_state_dict(torch.load(model_path))
model.eval()

# Prepare the tokenized input for BERT
tweets_normalized = [normalize_text(tweet) for tweet in tweets]
encoded_inputs = tokenizer(tweets_normalized, padding='longest', truncation=True, return_tensors='pt')
input_ids = encoded_inputs['input_ids']
attention_mask = encoded_inputs['attention_mask']

# Use the BERT model to make predictions
with torch.no_grad():
    logits = model(input_ids, attention_mask=attention_mask).logits

# Convert logits to probabilities using sigmoid
probs = torch.sigmoid(logits).cpu().numpy()

# Define toxicity ranges and the next range
toxicity_ranges = {
    0: (0, 25),    # label 0: Non-Offensive
    1: (25, 50),   # label 1: Non-Offensive but with Expletive Language
    2: (50, 100),  # label 2: Offensive Targeting a Group
    3: (50, 100)   # label 3: Offensive Targeting an Individual
}

toxicity_ranges_next = {
    0: (0, 10)   # label 0: Offensive Targeting an Individual
}

# Initialize a list to store toxicity percentages
toxicity_percentages = []

# Process each tweet's probabilities
for prob in probs:
    top_indices = np.argsort(prob)[-2:]
    label = np.argmax(prob)  # Get the predicted label
    toxicity_range = toxicity_ranges[label]  # Get the toxicity range for the predicted label
    mapped_probability = np.interp(prob[label], [0, 1], toxicity_range)
    mapped_probability_next = np.interp(prob[top_indices[0]], [0, 1], (0, 10))
    toxicity_percentage = (mapped_probability - mapped_probability_next)
    toxicity_percentages.append(toxicity_percentage)

total_toxicity_score = np.mean(toxicity_percentages)
total_toxicity_score= np.mean(toxicity_percentages)
print("Total Toxicity Percentage BERT Multi: ",total_toxicity_score)
print("----------------------------------------------------\n")
# Define the output file name
output_file = "toxicity_results_bert_multi_30A_eldiariodedross.txt"

# Open the file for writing
# Open the file for writing with 'utf-8' encoding
with open(output_file, "w", encoding="utf-8") as f:
    f.write(f"Total Toxicity Percentage: {total_toxicity_score}\n")
    f.write("----------------------------------------------------\n")
    for i, tweet in enumerate(tweets):
        label = np.argmax(probs[i])  # Get the predicted label for this tweet
        f.write(f"Tweet: {tweet}\n")
        f.write(f"Predicted Label: {label}\n")
        f.write(f"Toxicity Percentages: {toxicity_percentages[i]}%\n")
        f.write("----------------------------------------------------\n")

print("Results saved to", output_file)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total Toxicity Percentage BERT Multi:  40.107630245387554
----------------------------------------------------

Results saved to toxicity_results_bert_multi_30A_eldiariodedross.txt


# Comparison of BERTS

In [145]:
import pandas as pd

# Load the CSV files into Pandas DataFrames
csv_file1 = "toxicity_resultsbert_spanish_28A_eldiariodedross.csv"
csv_file2 = "toxicity_results.csv"

df1 = pd.read_csv(csv_file1, encoding='utf-8')
df2 = pd.read_csv(csv_file2, encoding='utf-8')

# Compare the "predicted Label" column between the two DataFrames
mismatched_rows = df1[df1["predicted Label"] != df2["predicted Label"]]
print(f"Number of Mismatched Tweets: {len(mismatched_rows)}")
# Display the mismatched tweets with their labels and toxicity percentages
print("Mismatched Tweets:")
for index, row in mismatched_rows.iterrows():
    print("Tweet:", row["tweet"])
    print("Label (File 1):", row["predicted Label"])
    print("Toxicity Percentage (File 1):", row["toxicity percentage"])
    
    # Find the corresponding row in the second DataFrame
    matching_row_in_file2 = df2[(df2["tweet"] == row["tweet"]) & (df2["predicted Label"] != row["predicted Label"])]

    if not matching_row_in_file2.empty:
        print("Label (File 2):", matching_row_in_file2.iloc[0]["predicted Label"])
        print("Toxicity Percentage (File 2):", matching_row_in_file2.iloc[0]["toxicity percentage"])
    
    print("------------------------")



Number of Mismatched Tweets: 41
Mismatched Tweets:
Tweet: No necesitas la aprobación ni la admiración de nadie. Y mucho menos de gente que está por debajo de ti.
Label (File 1): 2
Toxicity Percentage (File 1): 81.60171031951904
Label (File 2): 0
Toxicity Percentage (File 2): 19.968879520893097
------------------------
Tweet: Di algo que no puedas decir abiertamente, pero que en el fondo sabes que realmente no genera repudio sino que todo el mundo lo piensa...
Label (File 1): 3
Toxicity Percentage (File 1): 83.8571959733963
Label (File 2): 0
Toxicity Percentage (File 2): 21.43163114786148
------------------------
Tweet: Calificativos como "homófobo", "racista", "nazi" han perdido mucha fuerza... y no es culpa de la ultraderecha...
Label (File 1): 2
Toxicity Percentage (File 1): 78.35801362991333
Label (File 2): 0
Toxicity Percentage (File 2): 12.077334225177763
------------------------
Tweet: Quienes me amenazaban con: "SI VUELVES A TUITEAR ESO TU VIDA SE TERMINA!" hoy imploran: "Dross 

In [147]:
import pandas as pd

# Load the CSV files into Pandas DataFrames
csv_file1 = "toxicity_resultsbert_spanish_28A_eldiariodedross.csv"
csv_file2 = "toxicity_results.csv"

df1 = pd.read_csv(csv_file1, encoding='utf-8')
df2 = pd.read_csv(csv_file2, encoding='utf-8')

# Function to check if labels do not match as specified
def labels_do_not_match(label1, label2):
    label_pair = (label1, label2)
    # Specify the label pairs that are considered mismatches
    mismatched_label_pairs = [(3, 0), (3, 1), (2, 0), (2, 1), (0, 3), (1, 3), (0, 2), (1, 2)]
    return label_pair in mismatched_label_pairs

# Identify rows where labels do not match
mismatched_rows = df1[df1.apply(lambda row: labels_do_not_match(row["predicted Label"], df2[df2["tweet"] == row["tweet"]]["predicted Label"].values[0]), axis=1)]

# Display the count of rows where labels do not match
print(f"Number of Rows with Labels that Do Not Match: {len(mismatched_rows)}")

# Display the rows where labels do not match and their labels and toxicity percentages
print("Rows with Labels that Do Not Match:")
for index, row in mismatched_rows.iterrows():
    print("Tweet:", row["tweet"])
    print("Label (BERT spanish):", row["predicted Label"])
    print("Toxicity Percentage (BERT spanish):", row["toxicity percentage"])
    
    # Find the corresponding row in the second DataFrame
    matching_row_in_file2 = df2[(df2["tweet"] == row["tweet"]) & (df2.apply(lambda r: labels_do_not_match(row["predicted Label"], r["predicted Label"]), axis=1))]
    if not matching_row_in_file2.empty:
        print("Label (BERT multi):", matching_row_in_file2.iloc[0]["predicted Label"])
        print("Toxicity Percentage (BERT multi):", matching_row_in_file2.iloc[0]["toxicity percentage"])
    
    print("------------------------")


Number of Rows with Labels that Do Not Match: 24
Rows with Labels that Do Not Match:
Tweet: No necesitas la aprobación ni la admiración de nadie. Y mucho menos de gente que está por debajo de ti.
Label (BERT spanish): 2
Toxicity Percentage (BERT spanish): 81.60171031951904
Label (BERT multi): 0
Toxicity Percentage (BERT multi): 19.968879520893097
------------------------
Tweet: Di algo que no puedas decir abiertamente, pero que en el fondo sabes que realmente no genera repudio sino que todo el mundo lo piensa...
Label (BERT spanish): 3
Toxicity Percentage (BERT spanish): 83.8571959733963
Label (BERT multi): 0
Toxicity Percentage (BERT multi): 21.43163114786148
------------------------
Tweet: Calificativos como "homófobo", "racista", "nazi" han perdido mucha fuerza... y no es culpa de la ultraderecha...
Label (BERT spanish): 2
Toxicity Percentage (BERT spanish): 78.35801362991333
Label (BERT multi): 0
Toxicity Percentage (BERT multi): 12.077334225177763
------------------------
Tweet: A

In [148]:
import pandas as pd

# Define the CSV files
csv_file1 = "toxicity_resultsbert_spanish_28A_eldiariodedross.csv"
csv_file2 = "toxicity_results.csv"

# Load CSV files into Pandas DataFrames
df1 = pd.read_csv(csv_file1)
df2 = pd.read_csv(csv_file2)

# Get the unique labels in each DataFrame
unique_labels_df1 = df1['predicted Label'].unique()
unique_labels_df2 = df2['predicted Label'].unique()

# Count the occurrences of each label in each DataFrame
label_counts_df1 = df1['predicted Label'].value_counts()
label_counts_df2 = df2['predicted Label'].value_counts()

# Print the label counts for each CSV
print("Label counts for BERT Spanish")
for label, count in label_counts_df1.items():
    print(f"{label}: {count}")

print("\nLabel counts for BERT Multilingual")
for label, count in label_counts_df2.items():
    print(f"{label}: {count}")


Label counts for BERT Spanish
0: 36
1: 34
2: 18
3: 12

Label counts for BERT Multilingual
0: 56
1: 22
2: 17
3: 5
